In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 9.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 21.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=997f7cb1c14ba52ff9f4991f9c8319e668243234d7d7c3fa637c161417b02435
  Stored in directory: /home/jovyan/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [10]:
from datetime import date
from dateutil.rrule import rrule, DAILY, MONTHLY
import os
# from tqdm import tnrange, tqdm_notebook
import pickle
import sqlalchemy
import datetime
import os
import pandas as pd
from multiprocessing import cpu_count

from pyspark.sql import SparkSession
from pyspark.sql.functions import date_format
# from textwrap import dedent
from airflow import DAG 
from airflow.operators.python import PythonOperator

from datetime import datetime, timedelta
from finance.stock import (crawl_price,crawl_bargin,crawl_benchmark,crawl_pe)
import time
from finance.process import test_database, date_range
import os
from airflow.providers.postgres.hooks.postgres import PostgresHook
from airflow.models import Variable
from hdfs import InsecureClient
import pyarrow.parquet as pq
import pandas as pd
import pyarrow as pa
import tempfile
import io

In [11]:
tablename='price'
crawl_func =crawl_price
datetime_object = datetime.strptime('20190107', '%Y%m%d')

dates = date_range(datetime_object, datetime.now())
date = dates[0]

print(f'Crawlar {tablename} {date}')

df = crawl_func(date)


Crawlar price 2019-01-07


In [15]:
spark = (
    SparkSession.builder
    .appName("SaveToParquet")
    # .master(f"local[{cores}]")  # 使用动态检测到的核心数
    .master("spark://spark-master:7077")
    .getOrCreate()
)
hdfs_path = f'hdfs://namenode:8020/datawarehouse/{tablename}'


new_data_df=df
new_data_df = new_data_df.sort_values(by=['date'])
new_data_df = spark.createDataFrame(new_data_df)

23/05/07 15:32:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
